In [2]:
! git clone https://github.com/WongKinYiu/yolov7.git

%cd yolov7

Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 30.40 MiB/s, done.
Resolving deltas: 100% (465/465), done.
/content/yolov7
cfg	detect.py  hubconf.py  models	  requirements.txt  tools	  utils
data	export.py  inference   paper	  scripts	    train_aux.py
deploy	figure	   LICENSE.md  README.md  test.py	    train.py


In [11]:
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint
import pandas as pd
import cv2
import numpy as np
import os
import random

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def load_model():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.load('/content/drive/My Drive/mini_project/models/yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [6]:
def run_inference(image):
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (768, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 768, 960])
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 768, 960])
    output, _ = model(image) # torch.Size([1, 45900, 57])
    return output, image
def plot_skeleton_kpts(im, kpts, steps, orig_shape=None):
    #Plot the skeleton and keypointsfor coco datatset
    palette = np.array([[255, 128, 0], [255, 153, 51], [255, 178, 102],
                        [230, 230, 0], [255, 153, 255], [153, 204, 255],
                        [255, 102, 255], [255, 51, 255], [102, 178, 255],
                        [51, 153, 255], [255, 153, 153], [255, 102, 102],
                        [255, 51, 51], [153, 255, 153], [102, 255, 102],
                        [51, 255, 51], [0, 255, 0], [0, 0, 255], [255, 0, 0],
                        [255, 255, 255]])

    skeleton = [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12],
                [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3],
                [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]

    pose_limb_color = palette[[9, 9, 9, 9, 7, 7, 7, 0, 0, 0, 0, 0, 16, 16, 16, 16, 16, 16, 16]]
    pose_kpt_color = palette[[16, 16, 16, 16, 16, 0, 0, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9]]
    radius = 5
    num_kpts = len(kpts) // steps
    
    #store min x, max x, min y, max y
    min_x = im.shape[1]
    min_y = im.shape[0]
    max_x = 0
    max_y = 0

    for kid in range(num_kpts):
        r, g, b = pose_kpt_color[kid]
        x_coord, y_coord = kpts[steps * kid], kpts[steps * kid + 1]
        if not (x_coord % 640 == 0 or y_coord % 640 == 0):
            if steps == 3:
                conf = kpts[steps * kid + 2]
                if conf < 0.5:
                    continue
            min_x = min(min_x, x_coord)
            min_y = min(min_y, y_coord)
            max_x = max(max_x, x_coord)
            max_y = max(max_y, y_coord)
            cv2.circle(im, (int(x_coord), int(y_coord)), radius, (int(r), int(g), int(b)), -1)
            
    return [min_x, max_x, min_y, max_y]

def visualize_output(output, image):
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    pose = np.full(nimg.shape, 255).astype('uint8')
    
    for idx in range(output.shape[0]):
        x = plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
        plot_skeleton_kpts(pose, output[idx, 7:].T, 3)
    return nimg, pose, x


In [7]:
dir_name = "/content/drive/My Drive/mini_project/dataset/cleaned"
classes = ["carrying", "threat", "normal"]
#labels        0     ,     1   ,     2

In [16]:
def transform_data():
    c = classes[0]
    
    dir_path = os.path.join(dir_name, c) # create path
    data_ls = os.listdir(dir_path)       # list of file name
    count = 0
    class_index = classes.index(c)       # class label

    # do for 3 classes and range of 500 each. Showing one as example here due to RAM shortage doing all at one go
    df = pd.DataFrame(columns=['label','name','points'])

    for i in range(1):           
        img_name = data_ls[i]       # take item from top of the list
        file_path = os.path.join(dir_path, img_name)
        print(file_path)
        img = cv2.imread(file_path) 
        output, image = run_inference(img)
        img, pose, points = visualize_output(output, image)

        print(points)
        print("done ", count)
        count += 1
        toSave = {}

        # depends on class set above
        toSave['label'] = 0

        toSave['name'] = img_name

        toSave['points'] = points

        df = df.append(toSave, ignore_index=True)

    df.to_csv('/content/drive/My Drive/mini_project/pose.csv', mode='a', header=False)

transform_data()

/content/drive/My Drive/mini_project/dataset/cleaned/carrying/A0260847X_20220830_threat_00070.38555_30.png
[571.3939819335938, 692.4073486328125, 211.8095245361328, 513.035400390625]
done  0
